In [17]:
import os
import numpy as np
import pandas as pd


In [18]:
# Dataset path is by default saved in datset_result/bci_dataset.npz
dataset_path = os.path.join('dataset_result', 'bci_dataset.npz')

# Load the numpy file containing the dataset
dataset = np.load(dataset_path)
features, labels = dataset['features'], dataset['labels'] # get features and labels

['yes' 'yes' 'yes' ... 'yes' 'yes' 'yes']
